In [ ]:
from openai import OpenAI
import json
api_key = ""
client = OpenAI(api_key=api_key)

class OpenAIChatbot:
    def __init__(self,des="You are helpful", model="gpt-3.5-turbo-0125",stateless = False):
        self.model = model
        self.inst = des
        self.messages = [{"role": "system", "content": self.inst}]
        self.stateless = stateless
        self.cumulative_tokens = 0
        self.pre_tokens = 0

    def reset_chat(self):
        self.messages = [{"role": "system", "content": self.inst}]

    def chat(self, user_message, temperature=1, max_tokens=150, frequency_penalty=0, presence_penalty=0,form = "text"):
        self.add_message("user", user_message)
        response = client.chat.completions.create(model=self.model,
                                                  messages=self.messages,
                                                  temperature=temperature,
                                                  max_tokens=max_tokens,
                                                  frequency_penalty=frequency_penalty,
                                                  presence_penalty=presence_penalty,
                                                  response_format={ "type": form })
        cur_token = response.usage.total_tokens
        self.pre_tokens = cur_token
        self.cumulative_tokens += cur_token
        
        assistant_message = response.choices[0].message.content
        if not self.stateless:
            self.add_message("assistant", assistant_message)
        else:
            self.reset_chat()
        print(user_message)
        print(assistant_message)
        return assistant_message
    
    def show_tokens(self):
        print(f"cur: {self.pre_tokens}, cumulated: {self.cumulative_tokens}") 
    def getTokens(self):
        return self.cumulative_tokens
    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

    def create_embedding(self, input_text, embedding_model="text-embedding-ada-002", encoding_format="float"):
        try:
            response = client.embeddings.create(
                model=embedding_model,
                input=input_text,
                encoding_format=encoding_format
            )
            return response["data"][0]["embedding"]
        except Exception as e:
            print(f"Error creating embedding: {e}")
            return None
    
    def get_chat_history(self):
        return self.messages


In [ ]:
def ancestors(node, parentDict):
    ancestors = []
    current_node = node

    while current_node in parentDict:
        parent = parentDict[current_node]
        ancestors.append(parent)
        current_node = parent
    return ancestors

def FindSub(ln,data,bot):
    j = json.loads(bot.chat('\n'.join(ln),form="json_object",temperature=0.9,max_tokens=500))
    return j

def FindSol(goal,data):
    res = ''
    s = f'Main Goal: {goal}'
    ancestor = ancestors(goal,data['parentDict'])
    if ancestor:
        s += f',chain of parent goals:{"->".join(ancestor)}'
    if not data['childDict'].get(goal):
        bot = OpenAIChatbot(des='given an objective, you will produce an detailed and concise plan to achieve this goal,straight to the point and nothing else, <120 words',stateless=True)
        res = bot.chat(s,form="text",temperature=0.8,max_tokens=2000)
        data['tokens'] += bot.getTokens()
    else:
        children = data['childDict'].get(goal) 
        for child in children:
            s += '\n'+ child + ':\n'
            s += data['data'][child] 
        bot = OpenAIChatbot(des='I am solving a problem from a bottom up manner.Given a main goal,the chain of parent goals of it,and several of the sub-goals of the main goal and corresonding plans,merge the plans and produce a coherent, concise and detailed plan to achieve the main goal,<1000 words',stateless=True)
        res = bot.chat(s,form="text",temperature=0.8,max_tokens=2000)
        data['tokens'] += bot.getTokens()
    # rev = revise(res,data)
    return res

def revise(s,data):
    bot = OpenAIChatbot(des='given a plan, improve it and make it more concise',stateless=True)
    chat = bot.chat(s,temperature=0.9,max_tokens=700)
    data['tokens'] += bot.getTokens()
    return chat
    

In [ ]:
import json

def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.loads(file.read())

def save_json(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

def has_grandparent(node, parent_dict):
    parent = parent_dict.get(node)
    return parent and parent_dict.get(parent) is not None

def divide_phase(data):
    if not data['divideQueue']:
        return
    transition_to_conquer = False
    current_level_nodes = data['divideQueue'].copy()
    data['divideQueue'].clear()
    bot = OpenAIChatbot(des='given one or several objectives,seperated by "next line", respond with a json. Divide each objectives into 2 to 3 sub-objectives(should be complete sentenses, no punctuation and period) that are concise and essential,use EXACT original objective name(DO NOT CHANGE ANYTHING) as key and list formed by corresponding sub-objectives as value, no more than 10 words each',stateless=True)
    children_j = FindSub(current_level_nodes,data,bot) 
    data['tokens'] += bot.getTokens()

    for parent_node in current_level_nodes:
        children = children_j[parent_node]
        data['childDict'][parent_node] = children
        for child in children:
            data['parentDict'][child] = parent_node
            if not has_grandparent(child, data['parentDict']):
                data['divideQueue'].append(child)
            else:
                transition_to_conquer = True
            data['conquerStack'].append(child)

    if transition_to_conquer:
        data['isDividing'] = False

def conquer_phase(data):
    if data['conquerStack']:
        current_node = data['conquerStack'].pop()
        data['data'][current_node] = FindSol(current_node,data)

def main(file_path):
    data = load_json(file_path)

    if data['isDividing']:
        divide_phase(data)
    else:
        conquer_phase(data)
    save_json(data, file_path)


In [ ]:
file_path = 'chatState3.txt'
emptyJ = {
    "isDividing": True,
    "divideQueue": [],
    "tokens": 0,
    "conquerStack": [],
    "childDict": {},
    "parentDict": {},
    "data": {}
}
def startup(topic):
    emptyJ['divideQueue'].append(topic)
    emptyJ["conquerStack"].append(topic)
    save_json(emptyJ,file_path)
startup('find out exactly how to create a program that can automate and enhance major parts of the process of formulating a complete plan that achieves a given main objective')

In [ ]:
for _ in range(4):
    main(file_path)

In [ ]:
data = load_json(file_path)
print(list(data['data'].items())[-1][1])
